In [0]:
df_customer= spark.table("jpm.om_silver.silver_customer")
df_tran=spark.table("jpm.om_silver.silver_transaction")

In [0]:
%python
df_joined = df_customer.join(
    df_tran, 
    df_customer.cust_id == df_tran.cust_id, 
    "inner"
).select(
    df_customer.cust_id,
    'name',
    'age',
    'gender',
    'birthday',
    'zip',
    df_customer.city,
    'start_date',
    'end_date',
    'txn_id',
    'date',
    'year',
    'month',
    'day',
    'expense_type',
    'amt'
)

In [0]:
df_joined.columns

In [0]:
df_joined.createOrReplaceTempView("silver_joined")

In [0]:
%sql
create or replace table jpm.om_gold.all_records as 
select * from silver_joined

In [0]:
%sql
create or replace view jpm.om_gold.city_totals as 
select city, sum(amt) as total_sum from jpm.om_gold.all_records group by all

In [0]:
%sql
select * from jpm.naval_gold.city_totals

In [0]:
# Customer summary 
# top 5 customers 

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

top_customers = spark.sql("""
SELECT cust_id, SUM(amt) AS total_amount
FROM silver_joined
GROUP BY cust_id
ORDER BY total_amount DESC
LIMIT 5
""")

top_customers = top_customers.withColumn(
    "rank", 
    F.row_number().over(Window.orderBy(F.desc("total_amount")))
)
display(top_customers)

In [0]:
top_customers.createOrReplaceTempView("top_customers_view")